# 可视化光流

In [ ]:
from torchvision.utils import flow_to_image
from einops import rearrange
import torchvision.transforms.functional as tf
def flow2img(flow,name):
    """
    flow: h w 2
    """
    flow = rearrange(flow,"h w c -> c h w")
    flow_im = flow_to_image(flow)
    image = tf.to_pil_image(flow_im)
    image.save(name)